In [1]:
# mnist 
# 28x28 pixels (matrix), grayscale 0-255
# 784 pixels - each pixel is input for NN
# 2 hidden layers
# 10 digits - 10 classes - 10 output units 
# one hot encoding
# softmax activation func
# mnist data is iterable and 2-tuple format

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

2025-09-16 18:25:02.487664: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-16 18:25:02.487688: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-16 18:25:02.487694: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-16 18:25:02.487846: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-16 18:25:02.487859: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255. # between 0 and 1
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# shuffle 
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# batch size 1 sgd
# batch size # samples single batch gd
# when batching we find avg loss
BATCH_SIZE = 75

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples) # single batch
test_data = test_data.batch(num_test_samples) # single batch

# makes validation_data iterator
validation_inputs, validation_targets = next(iter(validation_data))

input_size = 784
output_size = 10
hidden_layer_size = 75

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # transforms tensor into a vector
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5


2025-09-16 18:25:03.067187: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
/opt/anaconda3/envs/tf/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-09-16 18:25:03.336286: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


720/720 - 8s - 11ms/step - accuracy: 0.8982 - loss: 0.3524 - val_accuracy: 0.9452 - val_loss: 0.1846
Epoch 2/5
720/720 - 7s - 9ms/step - accuracy: 0.9504 - loss: 0.1659 - val_accuracy: 0.9598 - val_loss: 0.1370
Epoch 3/5
720/720 - 7s - 9ms/step - accuracy: 0.9604 - loss: 0.1297 - val_accuracy: 0.9675 - val_loss: 0.1172
Epoch 4/5
720/720 - 6s - 9ms/step - accuracy: 0.9665 - loss: 0.1085 - val_accuracy: 0.9675 - val_loss: 0.1083
Epoch 5/5
720/720 - 6s - 9ms/step - accuracy: 0.9706 - loss: 0.0941 - val_accuracy: 0.9700 - val_loss: 0.0991


In [4]:
test_loss, test_accuracy = model.evaluate(test_data)
test_loss, test_accuracy

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - accuracy: 0.9648 - loss: 0.1181


(0.11811891943216324, 0.9648000001907349)